In [1]:
import praw
import datetime
import csv
import pandas as pd

In [2]:
def extract_comment_texts(comment, comment_texts=[]):
    comment_texts.append(comment.body)  # Add the current comment's text
    if hasattr(comment, 'replies'):
        replies = comment.replies
        for reply in replies:
            if isinstance(reply, praw.models.MoreComments):
                continue
            extract_comment_texts(reply, comment_texts)
    return comment_texts



In [3]:
def extract_submission_data(url, r):
    submission_id = url.split("/")[6]
    submission = r.submission(id=submission_id)
    comment_texts = []
    submission.comments.replace_more(limit=0)  # Remove MoreComments for a flat comment list
    for top_level_comment in submission.comments:
        if hasattr(top_level_comment, 'body'):
            extract_comment_texts(top_level_comment, comment_texts)
    data = {
        'title': submission.title,
        'url': submission.url,
        'author': str(submission.author),
        'score': submission.score,
        'created': datetime.datetime.fromtimestamp(submission.created),
        'upvote_ratio': submission.upvote_ratio,
        'comments': comment_texts
    }
    return data



In [4]:
def save_data_to_csv(data, filename="reddit_post_data.csv"):
    headers = ['Title', 'URL', 'Author', 'Score', 'Created', 'Upvote Ratio', 'Comment']
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for comment in data['comments']:
            writer.writerow([data['title'], data['url'], data['author'], data['score'],
                             data['created'], data['upvote_ratio'], comment])
    print(f"Data saved to {filename}")




In [5]:
user_agent = 'praw_scrapper_1.0'
r = praw.Reddit(username="aryamansingh_01",
                password="MGHector@123",
                client_id="8DLeUw6itg-tBu7JS-7F0A",
                client_secret="-cRyC4b6uYURAWUilU2PbiGaeY2mkg",
                user_agent=user_agent)

# The URL of the Reddit post
reddit_url = "https://www.reddit.com/r/unpopularopinion/comments/r0vvet/gas_cars_are_better_than_electric_cars/"

# Extracting the data
post_data = extract_submission_data(reddit_url, r)

# Saving the data to a CSV file
save_data_to_csv(post_data)


Data saved to reddit_post_data.csv


In [6]:
df = pd.read_csv("reddit_post_data.csv")

# Keep only the 'Comment' column
comments_only = df[['Comment']]

# Save the cleaned data to a new CSV file
comments_only.to_csv("comments_only.csv", index=False)

print("Cleaned data saved to comments_only.csv")

Cleaned data saved to comments_only.csv
